In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [3]:
from client import Client
from server_algo import Server_with_Algorithm
from server import Server
from datasets import load_dataset, concatenate_datasets
import validation

# DASHSCOPE_API_KEY = os.getenv("DASHSCOPE_API_KEY")

In [ ]:
# 1.首先选择要进行实验的client
clients = [Client(vectorstore_path="common_sense_db_0.6b"),
            # Client(vectorstore_path="computer_science_coding_related_db_0.6b"),
            # Client(vectorstore_path="law_related_db_0.6b"), Client(vectorstore_path="medicine_related_db_0.6b")]
            Client(vectorstore_path="arxiv_db_0.6b"), Client(vectorstore_path="legalbench_db_0.6b"), 
            Client(vectorstore_path="pubmedqa_db")]
# 2.对每个client进行加载
for c in clients:
    c.load_vectorstore()

/home/weiwei/RAGnet/client.py:63: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(model_name=model_path,


Vectorstore common_sense_db_0.6b loaded.


In [5]:
# 3.创建server对象，这里需要选择模型，请预先在ollama上部署
# server = Server(model_name="qwen3:4b")
server = Server(model_name="qwen3:4b")

/home/weiwei/RAGnet/server.py:21: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  self.llm = ChatOllama(model=model_name)


In [6]:
# 4.加载测试数据集，这里以trivia_qa为例子，这里取前100个
from datasets import Dataset, DatasetDict, load_dataset
from pathlib import Path

root_dir = "./test_dataset/cais_mmlu"
parquet_files = []

# 递归查找所有符合的文件
for dirpath, dirnames, filenames in os.walk(root_dir):
    for filename in filenames:
        if filename == "validation-00000-of-00001.parquet":
            parquet_files.append(os.path.join(dirpath, filename))

print(f"找到 {len(parquet_files)} 个 validation parquet 文件")

# 加载所有文件为一个dataset列表
datasets_list = [load_dataset("parquet", data_files=path) for path in parquet_files]


找到 35 个 validation parquet 文件


In [7]:
# 提取每个 DatasetDict 中的 validation split
samples = [ds["train"] for ds in datasets_list]

# 合并所有 validation Dataset
merged_samples = concatenate_datasets(samples)

In [8]:
random_samples = merged_samples.shuffle(seed=42).select(range(100))

In [ ]:
# 5.进行评估，并生成csv文件
validation.evaluate_datasets(clients=clients, server=server, samples=random_samples, top_k=5,
                            output_csv="mmlu_results_group2_topk_5.csv", dataset_name="mmlu")

Processing question: During exercise, adrenaline secretion from the adrenal glands is stimulated by:
Question background: Choices:
0. increased plasma glucose.
1. increased plasma fatty acids.
2. increased plasma ACTH.
3. increased sympathetic nerve activity.

Which one is correct?
This is a multiple-choice question. You must choose the correct answer from the options 1, 2, 3, or 4.     Answer with only the number of the correct choice. Do not explain your answer or include any additional text. Just reply with: 1, 2, 3, or 4
Gold answers: 3
uncleaned answer: <think>
Okay, let's tackle this question. The user is asking about the secretion of adrenaline during exercise and what triggers it. The options given are probably related to the nervous system and the endocrine system. Let me recall what I know about the body's response to physical activity.

When someone exercises, the body needs more energy, so the sympathetic nervous system gets activated. This leads to the release of adrenalin